In [3]:
import urllib

import requests
import numpy as np
import pandas as pd
from wikipedia2vec import Wikipedia2Vec
from scipy.stats import beta
import torch
from torch.nn import Sequential, Linear, Sigmoid


In [58]:
class Bandit:
    def __init__(self,max_size = 5000000):
        self._max_size = max_size
        self._embedding = Wikipedia2Vec.load('enwiki_20180420_100d.pkl')
        self.classifier = Sequential(Linear(2,1),Sigmoid())
    def wiki_size(self,url):
        props = requests.head(url).headers
        return torch.Tensor([int(props['content-length']) / self._max_size])
    def wiki_relevance(self,url,subject):
        key_word = url.split('/')[-1].replace('_',' ')
        key_vec = torch.Tensor(self._embedding.get_entity_vector(key_word))
        word_vec = torch.Tensor(self._embedding.get_word_vector(subject))
        return (key_vec @ word_vec)[None]
    def representation(self,url,subject):
        return torch.stack([self.wiki_relevance(url,subject),self.wiki_size(url)],dim=-1)
    def evaluate(self,url,subject):
        return self.classifier(self.representation(url,subject))
        

In [59]:
bandit = Bandit()
lotr = 'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings'
bandit.evaluate(lotr,'elves')